In [15]:
# m8.predictions.read_df()
# context.portfolio
m16.data.read().head()

,avg_amount_0,avg_amount_20,avg_amount_5,date,instrument,pe_ttm_0,rank_avg_amount_0,rank_avg_amount_10,rank_avg_amount_5,rank_return_0,...,rank_return_5,return_10,return_20,return_5,avg_amount_0/avg_amount_5,avg_amount_5/avg_amount_20,rank_avg_amount_0/rank_avg_amount_5,rank_avg_amount_5/rank_avg_amount_10,rank_return_0/rank_return_5,rank_return_5/rank_return_10
0,4.205952e+08,5.348484e+08,3.043804e+08,2017-01-03,000001.SZA,6.885001,0.951926,0.926916,0.913352,0.368235,...,0.472824,0.990270,0.959162,1.008811,1.381808,0.569097,1.042234,0.985367,0.778799,1.343261
1,4.115034e+08,5.200046e+08,3.273086e+08,2017-01-04,000001.SZA,6.885001,0.925273,0.923623,0.919589,0.152910,...,0.299079,0.995652,0.968288,1.004386,1.257234,0.629434,1.006181,0.995632,0.511270,1.073704
2,3.157697e+08,5.078230e+08,3.391389e+08,2017-01-05,000001.SZA,6.892518,0.883639,0.910740,0.921714,0.618997,...,0.416253,1.006586,0.966280,1.009912,0.931093,0.667829,0.958691,1.012051,1.487071,1.078037
3,3.271764e+08,5.012082e+08,3.428518e+08,2017-01-06,000001.SZA,6.862452,0.888459,0.911222,0.915518,0.606904,...,0.432059,0.996725,0.963080,1.007726,0.954279,0.684051,0.970444,1.004715,1.404677,0.941368
4,3.299946e+08,4.865385e+08,3.466537e+08,2017-01-09,000001.SZA,6.877485,0.901126,0.911285,0.911307,0.429778,...,0.363154,1.001094,0.961134,1.007709,0.951943,0.712490,0.988828,1.000025,1.183460,0.824419


In [11]:
# 本代码由可视化策略环境自动生成 2021年12月21日 18:36
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# 回测引擎：初始化函数，只执行一次
def m19_initialize_bigquant_run(context):
    # 加载预测数据
    context.ranker_prediction = context.options['data'].read_df()

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    # 预测数据，通过options传入进来，使用 read_df 函数，加载到内存 (DataFrame)
    # 设置买入的股票数量，这里买入预测股票列表排名靠前的5只
    stock_count = 5
    # 每只的股票的权重，如下的权重分配会使得靠前的股票分配多一点的资金，[0.339160, 0.213986, 0.169580, ..]
    context.stock_weights = T.norm([1 / math.log(i + 2) for i in range(0, stock_count)])
    # 设置每只股票占用的最大资金比例
    context.max_cash_per_instrument = 0.2
    context.options['hold_days'] = 5

# 回测引擎：每日数据处理函数，每天执行一次
def m19_handle_data_bigquant_run(context, data):
    # 按日期过滤得到今日的预测数据
    ranker_prediction = context.ranker_prediction[context.ranker_prediction.date == data.current_dt.strftime('%Y-%m-%d')]

    # 1. 资金分配
    # 平均持仓时间是hold_days，每日都将买入股票，每日预期使用 1/hold_days 的资金
    # 实际操作中，会存在一定的买入误差，所以在前hold_days天，等量使用资金；之后，尽量使用剩余资金（这里设置最多用等量的1.5倍）
    is_staging = context.trading_day_index < context.options['hold_days'] # 是否在建仓期间（前 hold_days 天）
    cash_avg = context.portfolio.portfolio_value / context.options['hold_days']
    cash_for_buy = min(context.portfolio.cash, (1 if is_staging else 1.5) * cash_avg)
    cash_for_sell = cash_avg - (context.portfolio.cash - cash_for_buy)
    positions = {e.symbol: p.amount * p.last_sale_price
                 for e, p in context.portfolio.positions.items()}

    # 2. 生成卖出订单：hold_days天之后才开始卖出；对持仓的股票，按机器学习算法预测的排序末位淘汰
    if not is_staging and cash_for_sell > 0:
        equities = {e.symbol: e for e, p in context.portfolio.positions.items()}
        instruments = list(reversed(list(ranker_prediction.instrument[ranker_prediction.instrument.apply(lambda x: x in equities)])))

        for instrument in instruments:
            context.order_target(context.symbol(instrument), 0)
            cash_for_sell -= positions[instrument]
            if cash_for_sell <= 0:
                break

    # 3. 生成买入订单：按机器学习算法预测的排序，买入前面的stock_count只股票
    buy_cash_weights = context.stock_weights
    buy_instruments = list(ranker_prediction.instrument[:len(buy_cash_weights)])
    max_cash_per_instrument = context.portfolio.portfolio_value * context.max_cash_per_instrument
    for i, instrument in enumerate(buy_instruments):
        cash = cash_for_buy * buy_cash_weights[i]
        if cash > max_cash_per_instrument - positions.get(instrument, 0):
            # 确保股票持仓量不会超过每次股票最大的占用资金量
            cash = max_cash_per_instrument - positions.get(instrument, 0)
        if cash > 0:
            context.order_value(context.symbol(instrument), cash)

# 回测引擎：准备数据，只执行一次
def m19_prepare_bigquant_run(context):
    pass


m1 = M.instruments.v2(
    start_date='2017-01-01',
    end_date='2020-01-01',
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m2 = M.advanced_auto_labeler.v2(
    instruments=m1.data,
    label_expr="""# #号开始的表示注释
# 0. 每行一个，顺序执行，从第二个开始，可以使用label字段
# 1. 可用数据字段见 https://bigquant.com/docs/develop/datasource/deprecated/history_data.html
#   添加benchmark_前缀，可使用对应的benchmark数据
# 2. 可用操作符和函数见 `表达式引擎 <https://bigquant.com/docs/develop/bigexpr/usage.html>`_

# 计算收益：5日收盘价(作为卖出价格)除以明日开盘价(作为买入价格)
shift(close, -5) / shift(open, -1)

# 极值处理：用1%和99%分位的值做clip
clip(label, all_quantile(label, 0.01), all_quantile(label, 0.99))

# 将分数映射到分类，这里使用20个分类
all_wbins(label, 20)

# 过滤掉一字涨停的情况 (设置label为NaN，在后续处理和训练中会忽略NaN的label)
where(shift(high, -1) == shift(low, -1), NaN, label)
""",
    start_date='',
    end_date='',
    benchmark='000300.SHA',
    drop_na_label=True,
    cast_label_int=True
)

m3 = M.input_features.v1(
    features="""# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
return_5
return_10
return_20
avg_amount_0/avg_amount_5
avg_amount_5/avg_amount_20
rank_avg_amount_0/rank_avg_amount_5
rank_avg_amount_5/rank_avg_amount_10
rank_return_0
rank_return_5
rank_return_10
rank_return_0/rank_return_5
rank_return_5/rank_return_10
pe_ttm_0
"""
)

m15 = M.general_feature_extractor.v7(
    instruments=m1.data,
    features=m3.data,
    start_date='',
    end_date='',
    before_start_days=0
)

m16 = M.derived_feature_extractor.v3(
    input_data=m15.data,
    features=m3.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m7 = M.join.v3(
    data1=m2.data,
    data2=m16.data,
    on='date,instrument',
    how='inner',
    sort=False
)

m10 = M.chinaa_stock_filter.v1(
    input_data=m7.data,
    index_constituent_cond=['沪深300'],
    board_cond=['上证主板', '深证主板'],
    industry_cond=['全部'],
    st_cond=['正常'],
    delist_cond=['非退市'],
    output_left_data=False
)

m13 = M.dropnan.v1(
    input_data=m10.data
)

m4 = M.stock_ranker_train.v6(
    training_ds=m13.data,
    features=m3.data,
    learning_algorithm='排序',
    number_of_leaves=30,
    minimum_docs_per_leaf=1000,
    number_of_trees=20,
    learning_rate=0.1,
    max_bins=1023,
    feature_fraction=1,
    data_row_fraction=1,
    plot_charts=True,
    ndcg_discount_base=1,
    m_lazy_run=False
)

m9 = M.instruments.v2(
    start_date=T.live_run_param('trading_date', '2020-01-01'),
    end_date=T.live_run_param('trading_date', '2021-12-01'),
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m17 = M.general_feature_extractor.v7(
    instruments=m9.data,
    features=m3.data,
    start_date='',
    end_date='',
    before_start_days=0
)

m18 = M.derived_feature_extractor.v3(
    input_data=m17.data,
    features=m3.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m5 = M.chinaa_stock_filter.v1(
    input_data=m18.data,
    index_constituent_cond=['沪深300'],
    board_cond=['上证主板', '深证主板'],
    industry_cond=['全部'],
    st_cond=['正常'],
    delist_cond=['非退市'],
    output_left_data=False
)

m14 = M.dropnan.v1(
    input_data=m5.data
)

m8 = M.stock_ranker_predict.v5(
    model=m4.model,
    data=m14.data,
    m_lazy_run=False
)

m19 = M.trade.v4(
    instruments=m9.data,
    options_data=m8.predictions,
    start_date='',
    end_date='',
    initialize=m19_initialize_bigquant_run,
    handle_data=m19_handle_data_bigquant_run,
    prepare=m19_prepare_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark='000300.SHA'
)


[2021-12-21 18:34:35.637730] INFO: moduleinvoker: instruments.v2 开始运行..

[2021-12-21 18:34:35.648528] INFO: moduleinvoker: 命中缓存

[2021-12-21 18:34:35.650862] INFO: moduleinvoker: instruments.v2 运行完成[0.013139s].

[2021-12-21 18:34:35.657349] INFO: moduleinvoker: input_features.v1 开始运行..

[2021-12-21 18:34:35.667538] INFO: moduleinvoker: 命中缓存

...

[2021-12-21 18:34:35.696684] INFO: moduleinvoker: 命中缓存

[2021-12-21 18:34:35.699257] INFO: moduleinvoker: general_feature_extractor.v7 运行完成[0.011357s].

[2021-12-21 18:34:35.710772] INFO: moduleinvoker: derived_feature_extractor.v3 开始运行..

[2021-12-21 18:34:35.721493] INFO: moduleinvoker: 命中缓存

[2021-12-21 18:34:35.724138] INFO: moduleinvoker: derived_feature_extractor.v3 运行完成[0.013369s].